# HW3_Q4

In [1]:
import pandas as pd, numpy as np
from scipy.io import arff

In [2]:
data = arff.loadarff('veh-prime.arff')
df = pd.DataFrame(data[0])
df.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f27,f28,f29,f30,f31,f32,f33,f34,f35,CLASS
0,0.063,0.160,0.509,-0.967,0.058,0.000,0.874,0.271,1.307,-0.011,...,-0.924,-0.077,0.108,-0.003,0.381,-0.314,0.929,0.184,-0.001,b'noncar'
1,-0.037,-0.325,-0.626,-0.029,0.121,-0.409,-0.002,-0.835,-0.595,-0.253,...,0.270,0.533,0.152,-0.978,0.157,0.011,-0.254,0.453,-0.621,b'noncar'
2,-0.000,1.253,0.833,-0.970,1.516,0.014,-0.378,1.197,0.546,-0.402,...,-0.408,1.550,0.010,-0.652,-0.403,-0.151,0.000,0.049,-0.113,b'car'
3,-0.743,-0.082,-0.626,0.723,-0.006,-0.000,-0.080,-0.297,0.166,0.311,...,0.819,-0.077,-0.099,-0.001,-0.291,1.633,0.686,1.528,-0.000,b'noncar'
4,-0.939,-1.054,-0.140,0.036,-0.766,0.000,-0.272,1.077,5.236,-0.366,...,0.676,0.533,-0.003,0.122,-0.179,-1.449,0.024,-1.698,0.083,b'noncar'


In [3]:
df.shape

(846, 37)

In [4]:
X_un = df.iloc[:,:-1]
y_str = df.iloc[:,-1]
y = pd.get_dummies(y_str).iloc[:,:1]

y_clean = np.array(y.iloc[:,0])

len(y) == len(X_un) # check

True

# Z-score Normalize

In [5]:
X = (X_un - X_un.mean()) / X_un.std()

In [6]:
X.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35
0,0.122985,0.159990,0.509079,-2.539525,0.057933,0.054440,2.234571,0.270975,1.307035,-0.043243,...,-0.328991,-2.340529,-0.076986,0.299813,-0.059698,0.380980,-0.314029,2.463273,0.183995,0.066622
1,-0.142390,-0.325026,-0.626000,-0.081220,0.120933,-1.083403,0.004844,-0.835001,-0.594944,-0.687021,...,-0.061992,0.662641,0.532987,0.419432,-2.581561,0.156995,0.010974,-0.691686,0.452991,-1.573437
2,-0.044201,1.253025,0.833102,-2.547387,1.515941,0.093388,-0.952208,1.196956,0.546044,-1.083396,...,0.072008,-1.042676,1.549942,0.033388,-1.738354,-0.402969,-0.151027,-0.014290,0.048997,-0.229646
3,-2.015936,-0.082018,-0.626000,1.889617,-0.006067,0.054440,-0.193693,-0.297013,0.166048,0.813354,...,-1.263990,2.043495,-0.076986,-0.262941,-0.054525,-0.290976,1.632990,1.815213,1.527976,0.069268
4,-2.536071,-1.054049,-0.139966,0.089132,-0.766071,0.054440,-0.682401,1.076958,5.235993,-0.987627,...,7.283997,1.683819,0.532987,-0.001954,0.263618,-0.178984,-1.449040,0.049716,-1.697978,0.288824


# Functions 

### Majority Vote

In [7]:
def majority_vote(categories_list):
    
    """
    Takes in a list of votes of length k.
    Returns the value that consitutes the majority.
    """
    
    ones = categories_list.count(1)
    zeroes = categories_list.count(0)

    if ones > zeroes:
        result = 1
    else:
        result = 0
    
    return result

### Predict Instance

In [8]:
def predict_instance(X_train, y_train, test_instance, k):
    
    """
    Takes in training features, training targets, testing features,
    a value for k, and a value for n, an index value 
    that determines the testing instance against which 
    the training instances are measured in terms of Eucledian distance.
    
    Sorts the list of distances in ascending order,
    keeping only the first k of the them in a new list,
    with which it computes a majority vote. 
    
    Returns the result of the majority vote. 
    """
    
    distances = []
    targets = []
    
    # Get the distances for one instance
    test = np.array(y_train)
    train = np.array(X_train)
    instance = np.array(test_instance)
    
    #1st for loop
    for i in range(len(train)):
    
        dist = np.sqrt(np.sum(np.square(instance - train[i]))) # removed [n] from after instance
        distances.append([dist,i])
     
    distances = sorted(distances)
    
    #2nd for loop
    for i in range(k):
        index = distances[i][1]
        targets.append(y_train[index])
    
    # majority vote
    return majority_vote(targets)


### KNN_accuracy

In [9]:
def KNN_accuracy(pred_X, t_test):
    
    """
    Takes in predicted and actual values for test instances.
    Returns an accuracy score.
    """
    
    test_target = np.array(t_test)
    pred_target = np.array(pred_X)
    
    accuracy_score = 0
    
    for i in range(len(pred_X)):
        if test_target[i] == pred_target[i]:
            accuracy_score = accuracy_score + 1
        
    return (accuracy_score / len(pred_X))

# LOOCV (Leave-one-out Cross Validation)

In [10]:
def LOOCV(X,y,f):
    
    '''f for feature #, X for features; y for target column
    '''
    
    pred = []
    
    for i in range(len(y)):
        
        bad_ix = X.index.isin([i])
        
        X_swept = X[~bad_ix]
        y_swept = y[~bad_ix]
        
        y_clean = np.array(y.iloc[:,0])
        y_swept_clean = np.array(y_swept.iloc[:,0])

        Xtrain1 = X_swept.iloc[:,f] # z
        ytrain1 = y_swept_clean # all but z
        
        Xtest1 = X.iloc[i,f]
        ytest1 = y_clean[i] # all but z

        y_pred = predict_instance(Xtrain1, ytrain1, Xtest1, 7)
        
        pred.append(y_pred)
        
    return pred
    

### Wrapper Method

In [11]:
def wrapper_method(X,y):
    
    '''Takes in features (X) and target (y) 
    and iterates through all possible single-feature-based 
    predicitions of a KNN-based LOOCV, then returns sorted
    list of LOOCV accuracy scores followed by the number
    for each of the 36 features'''
    
    y_clean = np.array(y.iloc[:,0])
    accuracy = []
    for i in range(X.shape[1]):
        res = LOOCV(X,y,i)
        score = KNN_accuracy(res,y_clean)
        accuracy.append([score, i])
        
    return sorted(accuracy, reverse = True)

In [12]:
def wrapper_method_add(X,y,f):
    
    '''Takes in features (X) and target (y)
    and a list of pre-established features (f)
    and iterates through all possible predictions based on 
    the pre established feature list + a new single feature
    then returns sorted list of LOOCV accuracy scores
    followed by the number of the newest, additional feature'''
    
    y_clean = np.array(y.iloc[:,0])
    accuracy = []
    for i in range(X.shape[1]):
        feat = []
        feat.append(i) # add new feature to feat list 
        j = i
        for i in range(len(f)):
            feat.append(f[i])
        res = LOOCV(X,y,feat)
        score = KNN_accuracy(res,y_clean)
        accuracy.append([score, j])
        
    return sorted(accuracy, reverse = True)

In [13]:
def wrapper_method_old(X,y,f):
    
    y_clean = np.array(y.iloc[:,0])
    accuracy = []
    for i in range(X.shape[1]):
        res = LOOCV(X,y,[f,i])
        score = KNN_accuracy(res,y_clean)
        accuracy.append([score, i])
        
    return sorted(accuracy, reverse = True)

# 4.a)

### 1st run: optimal feature = 14

In [14]:
first_run = wrapper_method(X,y)

In [15]:
first_run

[[0.7635933806146572, 14],
 [0.7505910165484634, 20],
 [0.7411347517730497, 22],
 [0.7387706855791962, 13],
 [0.7222222222222222, 4],
 [0.710401891252955, 16],
 [0.6749408983451537, 2],
 [0.6572104018912529, 7],
 [0.6442080378250591, 19],
 [0.6418439716312057, 25],
 [0.6335697399527187, 1],
 [0.6134751773049646, 31],
 [0.599290780141844, 26],
 [0.5721040189125296, 8],
 [0.5697399527186762, 10],
 [0.5543735224586288, 32],
 [0.533096926713948, 29],
 [0.5319148936170213, 28],
 [0.5295508274231678, 34],
 [0.5283687943262412, 30],
 [0.5271867612293144, 15],
 [0.5189125295508275, 27],
 [0.5153664302600472, 5],
 [0.5141843971631206, 24],
 [0.5130023640661938, 33],
 [0.5130023640661938, 12],
 [0.5094562647754137, 9],
 [0.5070921985815603, 23],
 [0.5059101654846335, 21],
 [0.5047281323877069, 35],
 [0.5047281323877069, 18],
 [0.5011820330969267, 0],
 [0.49763593380614657, 17],
 [0.4858156028368794, 3],
 [0.4574468085106383, 11],
 [0.4574468085106383, 6]]

### 2nd run: optimal feature = 10

In [18]:
second_run = wrapper_method_add(X,y,[14])

In [19]:
second_run

[[0.8782505910165485, 10],
 [0.8274231678486997, 13],
 [0.8203309692671394, 4],
 [0.8167848699763594, 19],
 [0.8167848699763594, 8],
 [0.806146572104019, 2],
 [0.8014184397163121, 16],
 [0.7966903073286052, 26],
 [0.7777777777777778, 25],
 [0.7742316784869976, 20],
 [0.7742316784869976, 1],
 [0.7730496453900709, 32],
 [0.7683215130023641, 22],
 [0.7659574468085106, 34],
 [0.764775413711584, 31],
 [0.7635933806146572, 14],
 [0.7600472813238771, 7],
 [0.7411347517730497, 15],
 [0.7399527186761229, 30],
 [0.7399527186761229, 12],
 [0.7387706855791962, 11],
 [0.7375886524822695, 28],
 [0.7364066193853428, 24],
 [0.7340425531914894, 27],
 [0.7328605200945626, 35],
 [0.7293144208037825, 5],
 [0.7269503546099291, 29],
 [0.7269503546099291, 9],
 [0.7269503546099291, 0],
 [0.7257683215130024, 3],
 [0.7245862884160756, 17],
 [0.7222222222222222, 21],
 [0.7210401891252955, 18],
 [0.7198581560283688, 33],
 [0.7186761229314421, 6],
 [0.7163120567375887, 23]]

### 3rd run: optimal feature = 19

In [20]:
third_run = wrapper_method_add(X,y,[14,10])

In [21]:
third_run

[[0.9030732860520094, 19],
 [0.892434988179669, 2],
 [0.8806146572104019, 25],
 [0.8794326241134752, 31],
 [0.8782505910165485, 14],
 [0.8770685579196218, 10],
 [0.8770685579196218, 8],
 [0.875886524822695, 4],
 [0.8747044917257684, 16],
 [0.8699763593380615, 1],
 [0.8676122931442081, 22],
 [0.8676122931442081, 13],
 [0.8652482269503546, 7],
 [0.8546099290780141, 34],
 [0.8546099290780141, 28],
 [0.8546099290780141, 20],
 [0.8498817966903073, 32],
 [0.8416075650118203, 26],
 [0.8416075650118203, 11],
 [0.8404255319148937, 27],
 [0.8368794326241135, 24],
 [0.8356973995271868, 3],
 [0.83451536643026, 35],
 [0.8274231678486997, 15],
 [0.8250591016548463, 18],
 [0.8238770685579196, 9],
 [0.8238770685579196, 5],
 [0.8226950354609929, 17],
 [0.8215130023640662, 21],
 [0.8215130023640662, 12],
 [0.8215130023640662, 0],
 [0.817966903073286, 23],
 [0.8156028368794326, 6],
 [0.8120567375886525, 33],
 [0.8096926713947991, 29],
 [0.8073286052009456, 30]]

### 4th run: optimal feature = 8

In [22]:
fourth_run = wrapper_method_add(X,y,[14,10,19])

In [23]:
fourth_run

[[0.9361702127659575, 8],
 [0.9231678486997635, 25],
 [0.9196217494089834, 4],
 [0.91725768321513, 7],
 [0.9160756501182034, 1],
 [0.9148936170212766, 32],
 [0.9125295508274232, 16],
 [0.9113475177304965, 2],
 [0.9101654846335697, 20],
 [0.9078014184397163, 26],
 [0.9054373522458629, 28],
 [0.9054373522458629, 22],
 [0.9054373522458629, 13],
 [0.9018912529550828, 14],
 [0.8995271867612293, 10],
 [0.8983451536643026, 34],
 [0.8971631205673759, 19],
 [0.8817966903073287, 27],
 [0.8806146572104019, 24],
 [0.8806146572104019, 18],
 [0.8806146572104019, 17],
 [0.8806146572104019, 0],
 [0.8782505910165485, 15],
 [0.8782505910165485, 11],
 [0.875886524822695, 31],
 [0.875886524822695, 12],
 [0.875886524822695, 9],
 [0.8711583924349882, 30],
 [0.8699763593380615, 33],
 [0.8699763593380615, 3],
 [0.8687943262411347, 6],
 [0.8652482269503546, 29],
 [0.8640661938534279, 21],
 [0.8628841607565012, 5],
 [0.8617021276595744, 35],
 [0.859338061465721, 23]]

### 5th run: optimal feature = 7

In [24]:
fifth_run = wrapper_method_add(X,y,[14,10, 19, 8])

In [25]:
fifth_run

[[0.9562647754137116, 7],
 [0.9456264775413712, 26],
 [0.9444444444444444, 34],
 [0.9432624113475178, 1],
 [0.9408983451536643, 32],
 [0.9373522458628841, 25],
 [0.9349881796690307, 20],
 [0.9349881796690307, 16],
 [0.9349881796690307, 14],
 [0.933806146572104, 4],
 [0.9326241134751773, 13],
 [0.9314420803782506, 10],
 [0.9302600472813238, 22],
 [0.9302600472813238, 2],
 [0.9290780141843972, 19],
 [0.925531914893617, 28],
 [0.9243498817966903, 8],
 [0.9160756501182034, 31],
 [0.9101654846335697, 18],
 [0.9101654846335697, 0],
 [0.9089834515366431, 17],
 [0.9089834515366431, 11],
 [0.9089834515366431, 5],
 [0.9089834515366431, 3],
 [0.9066193853427896, 21],
 [0.9054373522458629, 35],
 [0.9042553191489362, 29],
 [0.9030732860520094, 27],
 [0.9030732860520094, 12],
 [0.9030732860520094, 9],
 [0.900709219858156, 23],
 [0.8959810874704491, 24],
 [0.8947990543735225, 30],
 [0.8947990543735225, 15],
 [0.8936170212765957, 33],
 [0.892434988179669, 6]]

### 6th run: optimal feature = 25

In [26]:
sixth_run = wrapper_method_add(X,y,[14,10, 19, 8, 7])

In [27]:
sixth_run

[[0.958628841607565, 25],
 [0.9550827423167849, 14],
 [0.9550827423167849, 10],
 [0.9539007092198581, 1],
 [0.9527186761229315, 7],
 [0.9515366430260047, 20],
 [0.9515366430260047, 13],
 [0.9515366430260047, 2],
 [0.950354609929078, 22],
 [0.950354609929078, 19],
 [0.950354609929078, 16],
 [0.9491725768321513, 34],
 [0.9491725768321513, 32],
 [0.9479905437352246, 4],
 [0.9456264775413712, 8],
 [0.9432624113475178, 26],
 [0.9314420803782506, 31],
 [0.9314420803782506, 28],
 [0.9314420803782506, 5],
 [0.9302600472813238, 35],
 [0.9290780141843972, 30],
 [0.9267139479905437, 27],
 [0.9267139479905437, 17],
 [0.9243498817966903, 11],
 [0.9231678486997635, 3],
 [0.9219858156028369, 21],
 [0.9219858156028369, 0],
 [0.9184397163120568, 15],
 [0.9184397163120568, 9],
 [0.9160756501182034, 24],
 [0.9148936170212766, 29],
 [0.9137115839243499, 33],
 [0.9137115839243499, 18],
 [0.9113475177304965, 6],
 [0.9089834515366431, 12],
 [0.9042553191489362, 23]]

### 7th run: optimal feature = 1

In [36]:
seventh_run = wrapper_method_add(X,y,[14,10, 19, 8, 7, 25])

In [37]:
seventh_run

[[0.9621749408983451, 1],
 [0.9609929078014184, 16],
 [0.9609929078014184, 14],
 [0.958628841607565, 4],
 [0.9562647754137116, 7],
 [0.9550827423167849, 19],
 [0.9550827423167849, 10],
 [0.9539007092198581, 13],
 [0.9539007092198581, 8],
 [0.9539007092198581, 2],
 [0.9515366430260047, 25],
 [0.9515366430260047, 20],
 [0.950354609929078, 22],
 [0.9491725768321513, 32],
 [0.9456264775413712, 26],
 [0.9444444444444444, 34],
 [0.9349881796690307, 28],
 [0.9302600472813238, 35],
 [0.9290780141843972, 11],
 [0.9278959810874704, 31],
 [0.9278959810874704, 15],
 [0.9267139479905437, 3],
 [0.925531914893617, 30],
 [0.9243498817966903, 33],
 [0.9243498817966903, 17],
 [0.9231678486997635, 18],
 [0.9208037825059102, 29],
 [0.9208037825059102, 0],
 [0.9196217494089834, 12],
 [0.9196217494089834, 9],
 [0.91725768321513, 27],
 [0.91725768321513, 21],
 [0.9148936170212766, 6],
 [0.9148936170212766, 5],
 [0.9137115839243499, 24],
 [0.9125295508274232, 23]]

### 8th run: optimal feature = N/A

We stop the greedy algorithm because the first available feature for selection—that is, the features that weren't previously included (7,19,8,25)—is feature 22, the addition of which generates a lower LOOCV accuracy score than the score generated in the previous run 

Since 96.22 > 95.98 we halt the Wrapper Method Forward Selection Process

In [41]:
eighth_run = wrapper_method_add(X,y,[14,10, 19, 8, 7, 25, 1])

In [42]:
eighth_run

[[0.9645390070921985, 7],
 [0.9609929078014184, 19],
 [0.9609929078014184, 8],
 [0.9598108747044918, 25],
 [0.9598108747044918, 22],
 [0.9598108747044918, 20],
 [0.958628841607565, 16],
 [0.958628841607565, 10],
 [0.958628841607565, 2],
 [0.9574468085106383, 13],
 [0.9562647754137116, 14],
 [0.9562647754137116, 4],
 [0.9539007092198581, 1],
 [0.9527186761229315, 34],
 [0.950354609929078, 32],
 [0.9456264775413712, 26],
 [0.9432624113475178, 28],
 [0.9432624113475178, 11],
 [0.9373522458628841, 15],
 [0.9361702127659575, 35],
 [0.9349881796690307, 12],
 [0.9326241134751773, 31],
 [0.9314420803782506, 33],
 [0.9314420803782506, 9],
 [0.9314420803782506, 3],
 [0.9243498817966903, 30],
 [0.9243498817966903, 29],
 [0.9231678486997635, 21],
 [0.9231678486997635, 18],
 [0.9208037825059102, 6],
 [0.9196217494089834, 24],
 [0.9196217494089834, 17],
 [0.9184397163120568, 0],
 [0.91725768321513, 23],
 [0.91725768321513, 5],
 [0.9148936170212766, 27]]